#### Nifty & BankNifty Yearly Returns

- Graph to visualise Nifty 50 and BankNifty yearly returns from 2015-2023
- Calculate metrics like variance, standard deviation, covariance, and correlation
- Track or compare the annual returns with annual inflation rate
- Total returns over the specified interval

In [ ]:
## Imports: Necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import date, datetime
import matplotlib.pyplot as plt

In [ ]:
## Creating a yearly dataframe - from an input dataframe
def create_yearly_dataframe(input_df):
    ## OHLC aggregator function
    ohlc_aggregator = {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last"
    }
    ## Resampling of dataframe based on financial year - from April - March
    yearly_df = input_df.resample("A-MAR", on="Date").agg(ohlc_aggregator)
    yearly_df.reset_index(inplace=True)
    return yearly_df

In [ ]:
## Reading of Input Files
BASE_PATH = "F:\\MyProjects\\StockStrategies\\SectorIndex_1min_Data"
nifty_df = pd.read_csv(f"{BASE_PATH}\\NIFTY 50_minute.csv", index_col=0)
banknifty_df = pd.read_csv(f"{BASE_PATH}\\NIFTY BANK_minute.csv", index_col=0)
## Converting the Date Column to a date-time index
nifty_df["Date"] = pd.to_datetime(nifty_df["Date"])
banknifty_df["Date"] = pd.to_datetime(banknifty_df["Date"])
## Slice the data upto recent March 31st - March 31, 2023
nifty_df = nifty_df[(nifty_df["Date"]<="2023-03-31")]
banknifty_df = banknifty_df[(banknifty_df["Date"]<="2023-03-31")]
## Preparing the yearly dataframe
nifty_yearly_df = create_yearly_dataframe(input_df=nifty_df)
bn_yearly_df = create_yearly_dataframe(input_df=banknifty_df)

In [ ]:
## Now add the column of change_percentage i.e. annual return for the year
nifty_yearly_df["Chg_pct"] = round((nifty_yearly_df["Close"].pct_change() * 100), 3)
bn_yearly_df["Chg_pct"] = round((bn_yearly_df["Close"].pct_change() * 100), 3)

In [ ]:
## Annual inflation data of India
inflation_data_list = [4.93, 4.60, 3.55, 3.38, 4.50, 6.11, 5.38, 6.77]

In [ ]:
## Create a merged dataframe, for plotting the bar graphs
merged_data = pd.merge(nifty_yearly_df[["Date","Chg_pct"]], bn_yearly_df[["Date","Chg_pct"]], on="Date", suffixes=("_Nifty", "_BankNifty"))
merged_data["Date"] = merged_data["Date"].dt.year
merged_data.set_index("Date", inplace=True)
merged_data.dropna(inplace=True)

In [ ]:
## Create a bar graph plot - of the annual return
plt.figure(figsize=(30, 10))
ax = merged_data.plot(kind='bar', width=0.85)
plt.plot(inflation_data_list, color="red", marker="o")
plt.title("Annual Returns of Nifty and Bank Nifty: 2015-2023")
plt.xlabel("Financial Year")
plt.ylabel("Annual Return (in percentage)")
plt.ylim(-60, 100)
plt.legend(['Inflation','Nifty', 'Bank Nifty'], loc="upper left")
ax.axhline(y=0, color='black', linestyle='-', label='Zero Line')
# Add labels with percentages inside the bars, vertically rotated
for p in ax.patches:
    height = p.get_height()
    if height >= 0:
        xytext = (0, 5)
        va = 'bottom'
    else:
        xytext = (0, -2)
        va = 'top'
    ax.annotate(f'{height:.1f}%', (p.get_x() + p.get_width() / 2., height),
                ha='center', va=va, fontsize=8, color='black', xytext=xytext,
                textcoords='offset points', rotation=90)
## To display the bar chart
plt.show()

In [ ]:
## Calculate different metrics of Nifty and Bank Nifty
## Set ddof=0 for population standard deviation
########## Variance ##########
nifty_variance = np.var(merged_data['Chg_pct_Nifty'], ddof=0)
banknifty_variance = np.var(merged_data['Chg_pct_BankNifty'], ddof=0)
########## Standard Deviation ##########
nifty_std_deviation = np.std(merged_data['Chg_pct_Nifty'], ddof=0)
banknifty_std_deviation = np.std(merged_data['Chg_pct_BankNifty'], ddof=0)
########## Covariance ##########
covariance = np.cov(merged_data['Chg_pct_Nifty'], merged_data['Chg_pct_BankNifty'], ddof=0)[0, 1]
########## Correlation ##########
correlation = np.corrcoef(merged_data['Chg_pct_Nifty'], merged_data['Chg_pct_BankNifty'])[0, 1]

In [ ]:
## Print out the results
print(f"Nifty Variance: {nifty_variance: .2f}")
print(f"BankNifty Variance: {banknifty_variance: .2f}")
print(f"Nifty Std Deviation: {nifty_std_deviation: .2f}")
print(f"BankNifty Std Deviation: {banknifty_std_deviation: .2f}")
print(f"CoVariance: {covariance: .2f}")
print(f"Correlation: {correlation: .2f}")

In [ ]:
## Whereas the cummulative sum of chg_pct of Nifty and Bank Nifty
print("Nifty: Total return (in percentage): ", round(merged_data["Chg_pct_Nifty"].sum(),2))
print("BankNifty: Total return (in percentage): ", round(merged_data["Chg_pct_BankNifty"].sum(),2))